##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Entrenamiento distribuido con DTensors


<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/dtensor_ml_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/distribute/dtensor_ml_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/distribute/dtensor_ml_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/distribute/dtensor_ml_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar cuaderno</a> </td>
</table>

## Visión general

DTensor ofrece una forma de distribuir el entrenamiento de su modelo entre dispositivos para mejorar la eficacia, la fiabilidad y la escalabilidad. Si desea más detalles sobre los conceptos de DTensor, consulte [La Guía de Programación de DTensor](https://www.tensorflow.org/guide/dtensor_overview).

En este tutorial, entrenará un modelo de Análisis de Opinión con DTensor. Con este ejemplo se demuestran tres esquemas de entrenamiento distribuido:

- Entrenamiento paralelo de datos, en el que las muestras de entrenamiento se fragmentan (particionan) en dispositivos.
- Entrenamiento paralelo del modelo, en el que las variables del modelo se fragmentan entre los dispositivos.
- Entrenamiento paralelo espacial, en el que las características de los datos de entrada se fragmentan en dispositivos (también conocido como [Particionamiento espacial](https://cloud.google.com/blog/products/ai-machine-learning/train-ml-models-on-large-images-and-3d-volumes-with-spatial-partitioning-on-cloud-tpus)).

La parte de entrenamiento de este tutorial está inspirada en el bloc de notas [Una guía Kaggle sobre Análisis de Opinión](https://www.kaggle.com/code/anasofiauzsoy/yelp-review-sentiment-analysis-tensorflow-tfds/notebook). Para conocer el flujo de trabajo completo de entrenamiento y evaluación (sin DTensor), consulte dicho bloc.

Este tutorial seguirá los siguientes pasos:

- Primero empiece con una limpieza de datos para obtener un `tf.data.Dataset` de frases tokenizadas y su polaridad.

- Luego construya un modelo MLP con capas personalizadas Dense y BatchNorm. Use un `tf.Module` para hacer un seguimiento de las variables de inferencia. El constructor del modelo toma argumentos adicionales `Layout` para controlar la fragmentación de las variables.

- Para el entrenamiento, primero usará el entrenamiento paralelo de datos junto con la función de punto de verificación de `tf.experimental.dtensor`. Después continúe con el Entrenamiento Paralelo del Modelo y el Entrenamiento Paralelo Espacial.

- La sección final describe brevemente la interacción entre `tf.saved_model` y `tf.experimental.dtensor` a partir de TensorFlow 2.9.


## Preparación

DTensor forma parte de la versión 2.9.0 de TensorFlow.

In [ ]:
!pip install --quiet --upgrade --pre tensorflow tensorflow-datasets

A continuación, importe `tensorflow` y `tensorflow.experimental.dtensor`. Luego, configure TensorFlow para que use 8 CPU virtuales.

Aunque este ejemplo usa CPUs, DTensor funciona igual en dispositivos con CPU, GPU o TPU.

In [ ]:
import tempfile
import numpy as np
import tensorflow_datasets as tfds

import tensorflow as tf

from tensorflow.experimental import dtensor
print('TensorFlow version:', tf.__version__)

In [ ]:
def configure_virtual_cpus(ncpu):
  phy_devices = tf.config.list_physical_devices('CPU')
  tf.config.set_logical_device_configuration(phy_devices[0], [
        tf.config.LogicalDeviceConfiguration(),
    ] * ncpu)

configure_virtual_cpus(8)
DEVICES = [f'CPU:{i}' for i in range(8)]

tf.config.list_logical_devices('CPU')

## Descargar el conjunto de datos

Descargue el conjunto de datos de críticas de IMDB para entrenar el modelo de análisis de sentimientos.

In [ ]:
train_data = tfds.load('imdb_reviews', split='train', shuffle_files=True, batch_size=64)
train_data

## Preparar los datos

Primero tokenize el texto. Aquí se usa una extensión de la codificación en un solo paso, el modo `'tf_idf'` de `tf.keras.layers.TextVectorization`.

- Para mayor rapidez, limite el número de tokens a 1200.
- Para conservar la sencillez de `tf.Module`, ejecute `TextVectorization` como paso de preprocesamiento antes del entrenamiento.

El resultado final de la sección de limpieza de datos es un `Dataset` con el texto tokenizado como `x` y la etiqueta como `y`.

**Nota**: Ejecutar `TextVectorization` como paso de preprocesamiento **no es una práctica habitual ni recomendable**, ya que hacerlo supone que los datos de entrenamiento caben en la memoria del cliente, lo que no siempre es así.


In [ ]:
text_vectorization = tf.keras.layers.TextVectorization(output_mode='tf_idf', max_tokens=1200, output_sequence_length=None)
text_vectorization.adapt(data=train_data.map(lambda x: x['text']))

In [ ]:
def vectorize(features):
  return text_vectorization(features['text']), features['label']

train_data_vec = train_data.map(vectorize)
train_data_vec

## Cree una red neuronal con DTensor

A continuación, construya una red Perceptrón Multicapa (MLP) con `DTensor`. La red usará capas Dense y BatchNorm totalmente conectadas.

`DTensor` expande TensorFlow utilizando la expansión multidatos de un solo programa (SPMD) de las operaciones regulares de TensorFlow según los atributos `dtensor.Layout` de sus `Tensor` y variables de entrada.

Las variables de las capas sensibles al `DTensor` son `dtensor.DVariable`, y los constructores de los objetos de capa sensibles al `DTensor` toman entradas adicionales `Layout` además de los parámetros habituales de las capas.

Nota: A partir de TensorFlow 2.9, las capas Keras como `tf.keras.layer.Dense`, y `tf.keras.layer.BatchNormalization` aceptan argumentos `dtensor.Layout`. Consulte el [Tutorial de Integración de DTensor en Keras](/tutorials/distribute/dtensor_keras_tutorial) para saber más sobre cómo usar Keras con DTensor.

### Capa Dense

La siguiente capa Dense personalizada define 2 variables de capa: $W_{ij}$ es la variable de las ponderaciones, y $b_i$ es la variable de los sesgos.

$$ y_j = \sigma(\sum_i x_i W_{ij} + b_j) $$


### Deducción de Layout

Este resultado surge de las siguientes observaciones:

- La fragmentación DTensor preferida para los operandos de una matriz producto punto $t_j = \sum_i x_i W_{ij}$ es fragmentar $\mathbf{W}$ y $\mathbf{x}$ de la misma forma a lo largo del eje $i$.

- La fragmentación DTensor preferida para los operandos de una suma de matrices $t_j + b_j$, es fragmentar $\mathbf{t}$ y $\mathbf{b}$ de la misma manera a lo largo del eje $j$.


In [ ]:
class Dense(tf.Module):

  def __init__(self, input_size, output_size,
               init_seed, weight_layout, activation=None):
    super().__init__()

    random_normal_initializer = tf.function(tf.random.stateless_normal)

    self.weight = dtensor.DVariable(
        dtensor.call_with_layout(
            random_normal_initializer, weight_layout,
            shape=[input_size, output_size],
            seed=init_seed
            ))
    if activation is None:
      activation = lambda x:x
    self.activation = activation
    
    # bias is sharded the same way as the last axis of weight.
    bias_layout = weight_layout.delete([0])

    self.bias = dtensor.DVariable(
        dtensor.call_with_layout(tf.zeros, bias_layout, [output_size]))

  def __call__(self, x):
    y = tf.matmul(x, self.weight) + self.bias
    y = self.activation(y)

    return y

### BatchNorm

Una capa de normalización por lotes ayuda a evitar el colapso de los modos mientas se entrena. En este caso, añadir capas de normalización por lotes ayuda a que el entrenamiento del modelo evite producir un modelo que sólo produzca ceros.

El constructor de la capa personalizada `BatchNorm` de abajo no toma un argumento `Layout`. Esto se debe a que `BatchNorm` no tiene variables de capa. Esto sigue funcionando con DTensor porque "x", la única entrada de la capa, ya es un DTensor que representa el lote global.

Nota: Con DTensor, el Tensor de entrada "x" siempre representa el lote global. Por lo tanto, `tf.nn.batch_normalization` se aplica al lote global. Esto es distinto del entrenamiento con `tf.distribute.MirroredStrategy`, en el que el Tensor 'x' sólo representa el fragmento por réplica del lote (el lote local).

In [ ]:
class BatchNorm(tf.Module):

  def __init__(self):
    super().__init__()

  def __call__(self, x, training=True):
    if not training:
      # This branch is not used in the Tutorial.
      pass
    mean, variance = tf.nn.moments(x, axes=[0])
    return tf.nn.batch_normalization(x, mean, variance, 0.0, 1.0, 1e-5)

Una capa de normalización por lotes completa (como `tf.keras.layers.BatchNormalization`) necesitará argumentos Layout para sus variables.

In [ ]:
def make_keras_bn(bn_layout):
  return tf.keras.layers.BatchNormalization(gamma_layout=bn_layout,
                                            beta_layout=bn_layout,
                                            moving_mean_layout=bn_layout,
                                            moving_variance_layout=bn_layout,
                                            fused=False)

### Juntando las capas

A continuación, construya una red perceptrón multicapa (MLP) con los bloques de construcción anteriores.  El diagrama siguiente muestra las relaciones de ejes entre la entrada `x` y las matrices de ponderación de las dos capas `Dense` sin aplicar ninguna fragmentación ni replicación de DTensores.

<img src="https://www.tensorflow.org/images/dtensor/no_dtensor.png" class="no-filter" alt="Las matrices de entrada y ponderación de un modelo no distribuido."> 


La salida de la primera capa `Dense` se pasa a la entrada de la segunda capa `Dense` (después de la `BatchNorm`). Por lo tanto, la fragmentación DTensor preferida para la salida de la primera capa `Dense` ($\mathbf{W_1}$) y la entrada de la segunda capa `Dense` ($\mathbf{W_2}$) es fragmentar $\mathbf{W_1}$ y $\mathbf{W_2}$$ del mismo modo a lo largo del eje común $\hat{j}$,

$$ \mathsf{Layout}[{W_{1,ij}}; i, j] = \left[\hat{i}, \hat{j}\right] \ \mathsf{Layout}[{W_{2,jk}}; j, k] = \left[\hat{j}, \hat{k} \right] $$

Aunque la deducción de disposición muestra que las 2 disposiciones no son independientes, en aras de la simplicidad de la interfaz del modelo, `MLP` tomará 2 argumentos `Layout`, uno por cada capa Dense.

In [ ]:
from typing import Tuple

class MLP(tf.Module):

  def __init__(self, dense_layouts: Tuple[dtensor.Layout, dtensor.Layout]):
    super().__init__()

    self.dense1 = Dense(
        1200, 48, (1, 2), dense_layouts[0], activation=tf.nn.relu)
    self.bn = BatchNorm()
    self.dense2 = Dense(48, 2, (3, 4), dense_layouts[1])

  def __call__(self, x):
    y = x
    y = self.dense1(y)
    y = self.bn(y)
    y = self.dense2(y)
    return y


Las APIs que usan DTensor suelen tener como punto de diseño el equilibrio entre la corrección de las restricciones de deducción del diseño y la sencillez de la API. También es posible capturar la dependencia entre las `Layout` con una API diferente. Por ejemplo, la clase `MLPStricter` crea los objetos `Layout` en el constructor.

In [ ]:
class MLPStricter(tf.Module):

  def __init__(self, mesh, input_mesh_dim, inner_mesh_dim1, output_mesh_dim):
    super().__init__()

    self.dense1 = Dense(
        1200, 48, (1, 2), dtensor.Layout([input_mesh_dim, inner_mesh_dim1], mesh),
        activation=tf.nn.relu)
    self.bn = BatchNorm()
    self.dense2 = Dense(48, 2, (3, 4), dtensor.Layout([inner_mesh_dim1, output_mesh_dim], mesh))


  def __call__(self, x):
    y = x
    y = self.dense1(y)
    y = self.bn(y)
    y = self.dense2(y)
    return y

Para asegurarse de que el modelo funciona, pruebe su modelo con diseños totalmente replicados y un lote totalmente replicado de entrada `'x'`.

In [ ]:
WORLD = dtensor.create_mesh([("world", 8)], devices=DEVICES)

model = MLP([dtensor.Layout.replicated(WORLD, rank=2),
             dtensor.Layout.replicated(WORLD, rank=2)])

sample_x, sample_y = train_data_vec.take(1).get_single_element()
sample_x = dtensor.copy_to_mesh(sample_x, dtensor.Layout.replicated(WORLD, rank=2))
print(model(sample_x))

## Mover datos al dispositivo

Normalmente, los iteradores `tf.data` (y otros métodos de extracción de datos) producen objetos tensor respaldados por la memoria del dispositivo anfitrión local. Estos datos deben transferirse a la memoria del dispositivo acelerador que respalda los tensores componentes del DTensor.

`dtensor.copy_to_mesh` no es adecuada para esta situación porque replica los tensores de entrada a todos los dispositivos por la perspectiva global de DTensor. Así que en este tutorial usará una función ayudante `repack_local_tensor`, para facilitar la transferencia de datos. Esta función ayudante usa `dtensor.pack` para enviar (y sólo enviar) el fragmento del lote global destinado a una réplica al dispositivo que respalda la réplica.

Esta función simplificada presupone un único cliente. En una aplicación multicliente, puede ser muy laborioso determinar la forma correcta de dividir el tensor local y el mapeado entre las piezas de la división y los dispositivos locales.

Está prevista una API DTensor adicional para simplificar la integración de `tf.data`, compatible con aplicaciones de cliente único y multicliente. Se lo haremos saber.

In [ ]:
def repack_local_tensor(x, layout):
  """Repacks a local Tensor-like to a DTensor with layout.

  This function assumes a single-client application.
  """
  x = tf.convert_to_tensor(x)
  sharded_dims = []

  # For every sharded dimension, use tf.split to split the along the dimension.
  # The result is a nested list of split-tensors in queue[0].
  queue = [x]
  for axis, dim in enumerate(layout.sharding_specs):
    if dim == dtensor.UNSHARDED:
      continue
    num_splits = layout.shape[axis]
    queue = tf.nest.map_structure(lambda x: tf.split(x, num_splits, axis=axis), queue)
    sharded_dims.append(dim)

  # Now we can build the list of component tensors by looking up the location in
  # the nested list of split-tensors created in queue[0].
  components = []
  for locations in layout.mesh.local_device_locations():
    t = queue[0]
    for dim in sharded_dims:
      split_index = locations[dim]  # Only valid on single-client mesh.
      t = t[split_index]
    components.append(t)

  return dtensor.pack(components, layout)

## Entrenamiento paralelo de datos

En esta sección, entrenará su modelo MLP haciendo un entrenamiento paralelo de datos. Las secciones siguientes mostrarán el entrenamiento paralelo del modelo y el entrenamiento paralelo espacial.

El entrenamiento en paralelo de datos es un esquema comúnmente usado para el aprendizaje automático distribuido:

- Las variables del modelo se reproducen en N dispositivos cada una.
- Un lote global se divide en N lotes por réplica.
- Cada lote por réplica se entrena en el dispositivo de réplica.
- El gradiente se reduce antes de que la ponderación de los datos se realice colectivamente en todas las réplicas.

El entrenamiento paralelo de datos ofrece una aceleración casi lineal con respecto al número de dispositivos.

### Crear una malla paralela de datos

Un típico bucle de entrenamiento en paralelismo de datos usa un DTensor `Mesh` que consta de una única dimensión `batch`, donde cada dispositivo se convierte en una réplica que recibe un fragmento del lote global.


<img src="https://www.tensorflow.org/images/dtensor/dtensor_data_para.png" class="no-filter" alt="Malla paralela de datos">

El modelo replicado se ejecuta en la réplica, por lo que las variables del modelo están totalmente replicadas (sin fragmentar).

In [ ]:
mesh = dtensor.create_mesh([("batch", 8)], devices=DEVICES)

model = MLP([dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh),
             dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh),])


### Empaquetar datos de entrenamiento en DTensores

El lote de datos de entrenamiento debe empaquetarse en DTensores fragmentados a lo largo del eje `'batch'`(primero), de modo que el DTensor distribuya uniformemente los datos de entrenamiento en la dimensión de malla `'batch'`.

**Nota**: En DTensor, el `batch size` siempre se refiere al tamaño de lote global. El tamaño del lote debe seleccionarse de modo que pueda dividirse uniformemente por el tamaño de la dimensión de malla `batch`.

In [ ]:
def repack_batch(x, y, mesh):
  x = repack_local_tensor(x, layout=dtensor.Layout(['batch', dtensor.UNSHARDED], mesh))
  y = repack_local_tensor(y, layout=dtensor.Layout(['batch'], mesh))
  return x, y

sample_x, sample_y = train_data_vec.take(1).get_single_element()
sample_x, sample_y = repack_batch(sample_x, sample_y, mesh)

print('x', sample_x[:, 0])
print('y', sample_y)

### Paso de entrenamiento

Este ejemplo usa un optimizador de Descenso Gradiente Estocástico con el Bucle de Entrenamiento Personalizado (CTL). Consulte la [Guía del Bucle de Entrenamiento Personalizado](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch) y el [Tutorial](https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough) para obtener más información sobre estos temas.

El `train_step` se encapsula como `tf.function` para indicar que este cuerpo se va a trazar como un grafo TensorFlow. El cuerpo de `train_step` consta de una pasada de inferencia hacia delante, una pasada de gradiente hacia atrás y la actualización de variables.

Observe que el cuerpo de `train_step` no contiene ninguna anotación DTensor especial. En su lugar, `train_step` sólo contiene operaciones TensorFlow de alto nivel que procesan la entrada `x` y `y` desde la vista global del lote de entrada y el modelo. Todas las anotaciones del DTensor (`Mesh`, `Layout`) se eliminan del paso de entrenamiento.

In [ ]:
# Refer to the CTL (custom training loop guide)
@tf.function
def train_step(model, x, y, learning_rate=tf.constant(1e-4)):
  with tf.GradientTape() as tape:
    logits = model(x)
    # tf.reduce_sum sums the batch sharded per-example loss to a replicated
    # global loss (scalar).
    loss = tf.reduce_sum(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=y))
  parameters = model.trainable_variables
  gradients = tape.gradient(loss, parameters)
  for parameter, parameter_gradient in zip(parameters, gradients):
    parameter.assign_sub(learning_rate * parameter_gradient)

  # Define some metrics
  accuracy = 1.0 - tf.reduce_sum(tf.cast(tf.argmax(logits, axis=-1, output_type=tf.int64) != y, tf.float32)) / x.shape[0]
  loss_per_sample = loss / len(x)
  return {'loss': loss_per_sample, 'accuracy': accuracy}

### Punto de verificación

Puede crear un punto de verificación de un modelo DTensor usando `tf.train.Checkpoint` que ya viene integrado. Guardar y restaurar DVariables fragmentadas realizará un guardado y restauración fragmentados eficientes. Actualmente, al usar `tf.train.Checkpoint.save` y `tf.train.Checkpoint.restore`, todas las DVariables deben estar en la misma malla de host, y las DVariables y las variables normales no pueden guardarse juntas. Puede obtener más información sobre los puntos de verificación en [esta guía](../../guide/checkpoint.ipynb).

Cuando se restaura un punto de verificación DTensor, las `Layout` de las variables pueden ser diferentes de cuando se guarda el punto de verificación. Es decir, el guardado de modelos DTensor es independiente del layout y de la malla, y sólo afecta a la eficacia del guardado fragmentado. Puede guardar un modelo DTensor con una malla y un diseño y restaurarlo con una malla y un diseño diferentes. Este tutorial usa esta función para continuar el entrenamiento en las secciones Entrenamiento en modelos paralelos y Entrenamiento en modelos espaciales paralelos.


In [ ]:
CHECKPOINT_DIR = tempfile.mkdtemp()

def start_checkpoint_manager(model):
  ckpt = tf.train.Checkpoint(root=model)
  manager = tf.train.CheckpointManager(ckpt, CHECKPOINT_DIR, max_to_keep=3)

  if manager.latest_checkpoint:
    print("Restoring a checkpoint")
    ckpt.restore(manager.latest_checkpoint).assert_consumed()
  else:
    print("New training")
  return manager


### Bucle de entrenamiento

Para el esquema de entrenamiento paralelo de datos, entrena por épocas e informa del progreso. 3 épocas son insuficientes para entrenar el modelo: una precisión del 50% es lo mismo que adivinar al azar.

Active la función de punto de verificación para que pueda retomar el entrenamiento más tarde. En la siguiente sección, cargará el punto de verificación y hará el entrenamiento con un esquema paralelo diferente.

In [ ]:
num_epochs = 2
manager = start_checkpoint_manager(model)

for epoch in range(num_epochs):
  step = 0
  pbar = tf.keras.utils.Progbar(target=int(train_data_vec.cardinality()), stateful_metrics=[])
  metrics = {'epoch': epoch}
  for x,y in train_data_vec:

    x, y = repack_batch(x, y, mesh)

    metrics.update(train_step(model, x, y, 1e-2))

    pbar.update(step, values=metrics.items(), finalize=False)
    step += 1
  manager.save()
  pbar.update(step, values=metrics.items(), finalize=True)

## Entrenamiento paralelo de modelo

Si cambia a una `Mesh` de 2 dimensiones, y fragmenta las variables del modelo a lo largo de la segunda dimensión de la malla, entonces el entrenamiento se convierte en Paralelo de modelo.

En el entrenamiento Paralelo de Modelo, cada réplica del modelo abarca varios dispositivos (2 en este caso):

- Hay 4 réplicas del modelo, y el lote de datos de entrenamiento se distribuye entre las 4 réplicas.
- Los 2 dispositivos de una misma réplica del modelo reciben datos de entrenamiento replicados.


<img src="https://www.tensorflow.org/images/dtensor/dtensor_model_para.png" class="no-filter" alt="Malla paralela de modelos"> 


In [ ]:
mesh = dtensor.create_mesh([("batch", 4), ("model", 2)], devices=DEVICES)
model = MLP([dtensor.Layout([dtensor.UNSHARDED, "model"], mesh), 
             dtensor.Layout(["model", dtensor.UNSHARDED], mesh)])

Como los datos de entrenamiento siguen fragmentados a lo largo de la dimensión del lote, puede reutilizar la misma función `repack_batch` que en el caso del entrenamiento Paralelo de Datos. DTensor replicará automáticamente el lote por réplica a todos los dispositivos dentro de la réplica a lo largo de la dimensión de malla `"model"`.

In [ ]:
def repack_batch(x, y, mesh):
  x = repack_local_tensor(x, layout=dtensor.Layout(['batch', dtensor.UNSHARDED], mesh))
  y = repack_local_tensor(y, layout=dtensor.Layout(['batch'], mesh))
  return x, y

A continuación, ejecute el bucle de entrenamiento. El bucle de entrenamiento reutiliza el mismo Gestor de puntos de verificación que el ejemplo de entrenamiento Paralelo de datos, y el código parece idéntico.

Puede seguir entrenando el modelo entrenado paralelo de datos en entrenamiento paralelo de modelos.

In [ ]:
num_epochs = 2
manager = start_checkpoint_manager(model)

for epoch in range(num_epochs):
  step = 0
  pbar = tf.keras.utils.Progbar(target=int(train_data_vec.cardinality()))
  metrics = {'epoch': epoch}
  for x,y in train_data_vec:
    x, y = repack_batch(x, y, mesh)
    metrics.update(train_step(model, x, y, 1e-2))
    pbar.update(step, values=metrics.items(), finalize=False)
    step += 1
  manager.save()
  pbar.update(step, values=metrics.items(), finalize=True)

## Entrenamiento paralelo espacial

Cuando se entrenan datos de muy alta dimensionalidad (por ejemplo, una imagen muy grande o un vídeo), puede ser conveniente fragmentarlos a lo largo de la dimensión de la característica. Esto se denomina [Partición espacial](https://cloud.google.com/blog/products/ai-machine-learning/train-ml-models-on-large-images-and-3d-volumes-with-spatial-partitioning-on-cloud-tpus), que se introdujo por primera vez en TensorFlow para el entrenamiento de modelos con grandes muestras de entrada en 3D.


<img src="https://www.tensorflow.org/images/dtensor/dtensor_spatial_para.png" class="no-filter" alt="Malla paralela espacial">

DTensor también es compatible con este caso. El único cambio necesario es crear una Malla que incluya una dimensión `feature`, y aplicar el correspondiente `Layout`.


In [ ]:
mesh = dtensor.create_mesh([("batch", 2), ("feature", 2), ("model", 2)], devices=DEVICES)
model = MLP([dtensor.Layout(["feature", "model"], mesh), 
             dtensor.Layout(["model", dtensor.UNSHARDED], mesh)])


Fragmente los datos de entrada a lo largo de la dimensión `feature` al empaquetar los tensores de entrada en DTensores. Puede hacerlo con una función de reempaquetado ligeramente distinta, `repack_batch_for_spt`, donde `spt` significa Entrenamiento Paralelo Espacial.

In [ ]:
def repack_batch_for_spt(x, y, mesh):
    # Shard data on feature dimension, too
    x = repack_local_tensor(x, layout=dtensor.Layout(["batch", 'feature'], mesh))
    y = repack_local_tensor(y, layout=dtensor.Layout(["batch"], mesh))
    return x, y

El entrenamiento paralelo espacial también puede continuar desde un punto de verificación creado con otros esquemas de entrenamiento paralelo.

In [ ]:
num_epochs = 2

manager = start_checkpoint_manager(model)
for epoch in range(num_epochs):
  step = 0
  metrics = {'epoch': epoch}
  pbar = tf.keras.utils.Progbar(target=int(train_data_vec.cardinality()))

  for x, y in train_data_vec:
    x, y = repack_batch_for_spt(x, y, mesh)
    metrics.update(train_step(model, x, y, 1e-2))

    pbar.update(step, values=metrics.items(), finalize=False)
    step += 1
  manager.save()
  pbar.update(step, values=metrics.items(), finalize=True)

## SavedModel y DTensor

La integración de DTensor y SavedModel aún está en desarrollo.

A partir de TensorFlow `2.11`, `tf.saved_model` puede guardar modelos DTensor fragmentados y replicados, y permite realizar un guardado fragmentado eficiente en distintos dispositivos de la malla. Sin embargo, después de guardar un modelo, se pierden todas las anotaciones del DTensor y las firmas guardadas sólo se pueden usar con Tensores normales, no con DTensores.

In [ ]:
mesh = dtensor.create_mesh([("world", 1)], devices=DEVICES[:1])
mlp = MLP([dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh), 
           dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh)])

manager = start_checkpoint_manager(mlp)

model_for_saving = tf.keras.Sequential([
  text_vectorization,
  mlp
])

@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def run(inputs):
  return {'result': model_for_saving(inputs)}

tf.saved_model.save(
    model_for_saving, "/tmp/saved_model",
    signatures=run)

A partir de TensorFlow 2.9.0, sólo se puede llamar a una firma cargada con un Tensor normal, o con un DTensor totalmente replicado (que se convertirá en un Tensor normal).

In [ ]:
sample_batch = train_data.take(1).get_single_element()
sample_batch

In [ ]:
loaded = tf.saved_model.load("/tmp/saved_model")

run_sig = loaded.signatures["serving_default"]
result = run_sig(sample_batch['text'])['result']

In [ ]:
np.mean(tf.argmax(result, axis=-1) == sample_batch['label'])

## ¿Qué sigue?

Este tutorial muestra cómo construir y entrenar un modelo MLP de análisis de opiniones con DTensor.

Mediante las primitivas `Mesh` y `Layout`, DTensor puede transformar un `tf.function` de TensorFlow en un programa distribuido adecuado para diversos esquemas de entrenamiento.

Para no producir un modelo sobreajustado en el mundo real, en una aplicación de aprendizaje automático deben aplicarse la evaluación y la validación cruzada. Las técnicas presentadas en este tutorial también pueden aplicarse para introducir paralelismo a la evaluación.

Elaborar un modelo con `tf.Module` desde cero es mucho trabajo, y si se reutilizan los bloques de construcción existentes, como las capas y las funciones ayudantes, se puede acelerar drásticamente el desarrollo del modelo. A partir de TensorFlow 2.9, todas las Capas Keras incluidas en `tf.keras.layers` aceptan diseños DTensor como argumentos, y pueden usarse para construir modelos DTensor. Incluso puede reutilizar directamente un modelo Keras con DTensor sin modificar la implementación del modelo. Consulte el [Tutorial de integración de DTensor Keras](https://www.tensorflow.org/tutorials/distribute/dtensor_keras_tutorial) para saber cómo usar DTensor Keras. 